### Normalization Layers

The basic idea behind these layers is to normalize the output of an activation layer to improve the convergence during training

In [1]:
!pip install -q -U tensorflow-addons

     |████████████████████████████████| 1.1MB 8.6MB/s 


In [2]:
import tensorflow as tf
import tensorflow_addons as tfa

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


### Group Normalization 

Group Normalization(GN) divides the channels of your inputs into smaller sub groups and normalizes these values based on their mean and variance. Since GN works on a single example this technique is batchsize independent.

GN experimentally scored closed to batch normalization in image classification tasks. It can be beneficial to use GN instead of Batch Normalization in case your overall batch_size is low, which would lead to bad performance of batch normalization

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Reshape((28,28,1), input_shape = (28,28)),
  tf.keras.layers.Conv2D(filters = 10, kernel_size = (3,3), data_format = "channels_last"),
  # Groupnorm Layer
  tfa.layers.GroupNormalization(groups = 5, axis = 3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation = 'relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation = 'softmax')
])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model.fit(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.5022 - accuracy: 0.8508


### Instance Normalization 

Instance Normalization is special case of group normalization where the group size is the same size as the channel size (or the axis size).

Experimental results show that instance normalization performs well on style transfer when replacing batch normalization. Recently, instance normalization has also been used as a replacement for batch normalization in GANs.

Applying InstanceNormalization after a Conv2D Layer and using a uniformed initialized scale and offset factor.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Reshape((28,28,1), input_shape = (28,28)),
  tf.keras.layers.Conv2D(filters = 10, kernel_size = (3,3), data_format = "channels_last"),
  # LayerNorm Layer
  tfa.layers.InstanceNormalization(axis = 3, 
                                   center = True, 
                                   scale = True,
                                   beta_initializer = "random_uniform",
                                   gamma_initializer = "random_uniform"),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation = 'relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation = 'softmax')
])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model.fit(x_test, y_test)